In [ ]:
"""
Implementation of the Triplet Loss baseline based on the original code available on
https://github.com/White-Link/UnsupervisedScalableRepresentationLearningTimeSeries
"""

import os
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

os.chdir("../") #Load from parent directory
from data_utils import Plots,gen_loader,load_datasets
from models import select_encoder
utils_plot=Plots()

In [ ]:
class TripletLoss(torch.nn.modules.loss._Loss):
    """
    Triplet loss for representations of time series. Optimized for training
    sets where all time series have the same length.
    Takes as input a tensor as the chosen batch to compute the loss,
    a PyTorch module as the encoder, a 3D tensor (`B`, `C`, `L`) containing
    the training set, where `B` is the batch size, `C` is the number of
    channels and `L` is the length of the time series, as well as a boolean
    which, if True, enables to save GPU memory by propagating gradients after
    each loss term, instead of doing it after computing the whole loss.
    The triplets are chosen in the following manner. First the size of the
    positive and negative samples are randomly chosen in the range of lengths
    of time series in the dataset. The size of the anchor time series is
    randomly chosen with the same length upper bound but the the length of the
    positive samples as lower bound. An anchor of this length is then chosen
    randomly in the given time series of the train set, and positive samples
    are randomly chosen among subseries of the anchor. Finally, negative
    samples of the chosen length are randomly chosen in random time series of
    the train set.
    @param compared_length Maximum length of randomly chosen time series. If
           None, this parameter is ignored.
    @param nb_random_samples Number of negative samples per batch example.
    @param negative_penalty Multiplicative coefficient for the negative sample
           loss.
    """
    def __init__(self, compared_length, nb_random_samples, negative_penalty):
        super(TripletLoss, self).__init__()
        self.compared_length = compared_length
        if self.compared_length is None:
            self.compared_length = np.inf
        self.nb_random_samples = nb_random_samples
        self.negative_penalty = negative_penalty

    def forward(self, batch, encoder, train, save_memory=False):
        batch_size = batch.size(0)
        train_size = train.size(0)
        length = min(self.compared_length, train.size(2))

        # For each batch element, we pick nb_random_samples possible random
        # time series in the training set (choice of batches from where the
        # negative examples will be sampled)
        samples = np.random.choice(
            train_size, size=(self.nb_random_samples, batch_size)
        )
        samples = torch.LongTensor(samples)

        # Choice of length of positive and negative samples
        length_pos_neg = self.compared_length
        # length_pos_neg = np.random.randint(1, high=length + 1)


        # We choose for each batch example a random interval in the time
        # series, which is the 'anchor'
        random_length = self.compared_length

        #print(length,random_length,batch_size)
        beginning_batches = np.random.randint(
            0, high=length - random_length + 1, size=batch_size
        )  # Start of anchors

        # The positive samples are chosen at random in the chosen anchors
        beginning_samples_pos = np.random.randint(
            0, high=random_length + 1, size=batch_size
        )  # Start of positive samples in the anchors
        # Start of positive samples in the batch examples
        beginning_positive = beginning_batches + beginning_samples_pos
        # End of positive samples in the batch examples
        end_positive = beginning_positive + length_pos_neg + np.random.randint(0,self.compared_length)

        # We randomly choose nb_random_samples potential negative samples for
        # each batch example
        beginning_samples_neg = np.random.randint(
            0, high=length - length_pos_neg + 1,
            size=(self.nb_random_samples, batch_size)
        )

        representation = encoder(torch.cat(
            [batch[
                j: j + 1, :,
                beginning_batches[j]: beginning_batches[j] + random_length
            ] for j in range(batch_size)]))  # Anchors representations        
        ##
        positive_representation = encoder(torch.cat(
            [batch[
                j: j + 1, :, end_positive[j] - length_pos_neg: end_positive[j]
            ] for j in range(batch_size)]
        ))  # Positive samples representations

        size_representation = representation.size(1)
        # Positive loss: -logsigmoid of dot product between anchor and positive
        # representations
        loss = -torch.mean(torch.nn.functional.logsigmoid(torch.bmm(
            representation.view(batch_size, 1, size_representation),
            positive_representation.view(batch_size, size_representation, 1)
        )))

        # If required, backward through the first computed term of the loss and
        # free from the graph everything related to the positive sample
        if save_memory:
            loss.backward(retain_graph=True)
            loss = 0
            del positive_representation
            torch.cuda.empty_cache()

        multiplicative_ratio = self.negative_penalty / self.nb_random_samples
        for i in range(self.nb_random_samples):
            # Negative loss: -logsigmoid of minus the dot product between
            # anchor and negative representations
            negative_representation = encoder(
                torch.cat([train[samples[i, j]: samples[i, j] + 1][
                    :, :,
                    beginning_samples_neg[i, j]:
                    beginning_samples_neg[i, j] + length_pos_neg
                ] for j in range(batch_size)])
            )
            loss += multiplicative_ratio * -torch.mean(
                torch.nn.functional.logsigmoid(-torch.bmm(
                    representation.view(batch_size, 1, size_representation),
                    negative_representation.view(
                        batch_size, size_representation, 1
                    )
                ))
            )
            # If required, backward through the first computed term of the loss
            # and free from the graph everything related to the negative sample
            # Leaves the last backward pass to the training procedure
            if save_memory and i != self.nb_random_samples - 1:
                loss.backward(retain_graph=True)
                loss = 0
                del negative_representation
                torch.cuda.empty_cache()

        return loss

In [ ]:
def epoch_run(data, encoder, device, window_size,batch_size, optimizer=None, train=True):
    
    comp_len = window_size
    if window_size==-1:
        comp_len = None
        
    if train:
        encoder.train()
    else:
        encoder.eval()
        
    encoder = encoder.to(device)
    loss_criterion = TripletLoss(compared_length=comp_len, nb_random_samples=10, negative_penalty=1)

    epoch_loss = 0
    dataset = torch.utils.data.TensorDataset(torch.tensor(data).float().to(device),
                                             torch.zeros((len(data),1)).to(device))
    
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)
    i = 0

    for x_batch,y in data_loader:
        
        loss = loss_criterion(x_batch.to(device), encoder, torch.tensor(data).float().to(device))
        epoch_loss += loss.item()
        
        i += 1
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    return epoch_loss/i

In [ ]:
def learn_encoder(n_cross_val,data_type,datasets,lr,window_size,tr_percentage,batch_size,
                  encoder_type,encoding_size,decay,n_epochs,suffix,device,device_ids,verbose,show_encodings):
        
    accuracies=[]
    for cv in range(n_cross_val):
        train_data,train_labels,test_data,test_labels = load_datasets(data_type,datasets,cv)

        #Save Location
        save_dir = './results/baselines/%s_tloss/%s/'%(datasets,data_type)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        save_file = str((save_dir +'encoding_%d_encoder_%d_checkpoint_%d%s.pth.tar')
               %(encoding_size,encoder_type, cv,suffix))
        
        if verbose:
            print('Saving at: ',save_file)
        
        #Models

        input_size = train_data.shape[1]
        encoder,_ = select_encoder(device,encoder_type,input_size,encoding_size)
        
        #Training init
        params = encoder.parameters()
        optimizer = torch.optim.Adam(params, lr=lr, weight_decay=decay)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, n_epochs, gamma=0.999)
        
        #Split/Shuffle train and val
        inds = list(range(len(train_data)))
        random.shuffle(inds)
        train_data = train_data[inds]
        n_train = int(tr_percentage*len(train_data))
        best_acc = 0
        best_loss = np.inf
        train_loss, val_loss = [], []

        #Train
        for epoch in range(n_epochs):
            epoch_loss = epoch_run(train_data[:n_train], encoder, device, window_size,batch_size,
                                   optimizer=optimizer, train=True)
            epoch_loss_val = epoch_run(train_data[n_train:], encoder, device, window_size,batch_size,
                                       optimizer=optimizer, train=False)
            #scheduler.step()
            
            if verbose:
                print('\nEpoch ', epoch)
                print('Train ===> Loss: ', epoch_loss)
                print('Validation ===> Loss: ', epoch_loss_val)
            
            train_loss.append(epoch_loss)
            val_loss.append(epoch_loss_val)
            
            if epoch_loss_val<best_loss:
                state = {
                    'epoch': epoch,
                    'encoder_state_dict': encoder.state_dict()
                }
                best_loss = epoch_loss_val
                torch.save(state, save_file)
                if verbose:
                    print('Saving ckpt')
                
        accuracies.append(best_acc)
        plt.figure()
        plt.plot(np.arange(n_epochs), train_loss, label="Train")
        plt.plot(np.arange(n_epochs), val_loss, label="Validation")
        plt.title("Tloss Unsupervised Loss")
        plt.legend()
        plt.savefig(save_dir +'encoding_%d_encoder_%d_checkpoint_%d%s.png'%(encoding_size,encoder_type, cv,suffix))
        if verbose:
            plt.show()
        plt.close()
        
        if verbose:
            print('Best Train ===> Loss: ', np.min(train_loss))
            print('Best Validation ===> Loss: ', np.min(val_loss))
    return

In [ ]:
def run_tloss(args):
    #Run Process

    if args['batch_size']<1:
        args['batch_size'] = max(1,int(min(len(args['train_data']),len(test_data))*args['batch_size']))
        print('Using batch_size:', args['batch_size'])
    
    learn_encoder(**args)
    
    #Plot Features
    title = 'Tloss Encoding TSNE for %s'%(args['data_type'])
    
    if args['show_encodings']:
        for cv in range(args['n_cross_val']):
            train_data,train_labels,test_data,test_labels = load_datasets(args['data_type'],args['datasets'],cv)
            utils_plot.plot_distribution(test_data, test_labels,args['encoder_type'],
                                                             args['encoding_size'],args['window_size'],'tloss',
                                                             args['datasets'],args['data_type'],args['suffix'],
                                                             args['device'], title, cv)
    return

In [ ]:
def main(args):
    
    #Devices
    args['device'] = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    args['device_ids'] = [i for i in range(torch.cuda.device_count())]
    print('Using', args['device'])
    
    
    if args['data_type'] == 'urban':
        args['n_cross_val'] = 10
        
    #Experiment Parameters
    args['window_size'] = 2500
    args['encoder_type'] = 1
    args['encoding_size'] = 128
    args['lr'] = 1e-3
    args['decay'] = 1e-5
    args['datasets'] = args['data_type']

    run_tloss(args)
    return

In [ ]:
args = {'n_cross_val':5,
        'data_type':'afdb', #options: afdb, ims, urban
        'tr_percentage':0.8,
        'n_epochs':100,
        'suffix':'',
        'batch_size':8,
        'show_encodings':False,
        'verbose': True} 

main(args)